还有一个问题：用户id是用来划分数据的，但是trace_id才是一一对应到原来的训练数据的诶。

划分完之后的数据，其实没必要再带上用户id了，还是得带上trace_id更重要一些。

以及，之前我从来都是丢掉用户id的。所以在处理实际上的原始数据的时候，我还得去哪里把用户id找回来才行。

In [14]:
itv = 200
for i in range(7):
    print(1390 - i*itv)

1390
1190
990
790
590
390
190


In [15]:
sorted_features = [i for i in range(1390)]
itv = 200
for i in range(7):
    if i == 0:
        test_feas = sorted_features
    else:
        test_feas = sorted_features[:-i*itv]
    
    ###########  
    if os.path.exists("trained_models/"+nm + f"-{len(test_feas)}"):
        continue
    ############
    
    print("*" * 30, len(test_feas))

****************************** 1390
****************************** 1190
****************************** 990
****************************** 790
****************************** 590
****************************** 390
****************************** 190


In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import time
import xgboost as xgb
import os, time
from tqdm import tqdm
import pyarrow.parquet as pq
import pyarrow as pa

import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

import sklearn
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

import tqdm, re, pickle, gc, os, time, random

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

from collections import defaultdict

In [2]:
def get_importance_fixed1(
    xgb_model_path,
    feas_cols_v2
):
    xgb_model = xgb.Booster()
    xgb_model.load_model(xgb_model_path)
    
    weights = xgb_model.get_score(importance_type="weight")
    gains = xgb_model.get_score(importance_type="gain")
    total_gains = {}
    for fea_avt in weights:
        total_gains[fea_avt] = weights[fea_avt]*gains[fea_avt]
        
    iptc_scores = total_gains ## xgb_model.get_score(importance_type="total_gain")
    importances = []
    for i in range(len(feas_cols_v2)):
        fea_avt = f"f{i}"
        if fea_avt not in iptc_scores:
            continue
        importances.append((feas_cols_v2[i], iptc_scores[fea_avt]))
    importances.sort(key=lambda x: x[1], reverse=True)
    sorted_features = [_[0] for _ in importances]
    
    return importances, sorted_features

def train_model_with_different_label_2(dtrain, dtest):
    
    params={
            'booster':'gbtree',
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'max_depth': 5,
            'scale_pos_weight': 3,
            'learning_rate': 0.1, 
            'reg_lambda': 5,
            'reg_alpha': 0, 
            'colsample_bytree': 0.8,
#             'tree_method': 'gpu_hist',
            "n_gpus": -1,
        }
    # params['nthread'] = 25
    train_start_time = time.time()
    booster_maidian = xgb.train(params, dtrain, num_boost_round=10, evals=[(dtrain, 'train'), (dtest, 'test')],
                                                      early_stopping_rounds=50,verbose_eval=100)
    train_end_time = time.time()
    
    print(f"train time: {train_end_time - train_start_time}")
    
    if not os.path.exists("trained_models"):
        os.makedirs("trained_models")
    return booster_maidian

def train_original_model_1(
    train, test,
    feas_cols_v2, 
    dt_name,
    model_dir,
    target_label,
):
    # print("starting...")
    all_train_matrix = xgb.DMatrix(
        train[feas_cols_v2].values, 
        train[target_label].values,
        feature_names=feas_cols_v2
    )

    all_test_matrix = xgb.DMatrix(
        test[feas_cols_v2].values, 
        test[target_label].values,
        feature_names=feas_cols_v2
    )
    # print("we are here...")
    model = train_model_with_different_label_2(
        all_train_matrix, all_test_matrix
    )
    del all_train_matrix, all_test_matrix
    gc.collect()
    # print("we are here now")
    model_path = os.path.join(model_dir, dt_name)
    
    model.save_model(model_path)
    
    return model_path

In [3]:
nm = "pre1"
target_label = "isDefault"
userid_col = "id" ## loan_account_id
traceid_col = "issueDate" ## trace_id
n_fold = 5
NEED_TO_PREPARE_DATA = True ## 用这个标记一下，是否需要重新准备数据
NEED_TO_TRAIN = True ## 用这个标记一下，是否需要重新准备数据

In [4]:
with open(f"preprocessedDataset/{nm}.txt", "r") as f:
    feas_cols_v2 = [i.strip() for i in f.readlines()]
len(feas_cols_v2)

49

In [5]:
## 这一步userid的获取要独立进行。
## 为的是好在实际环境中修改。
all_userid = pq.read_table(
    f"preprocessedDataset/{nm}.parquet",
    columns = [userid_col]
).to_pandas()
all_userid.shape

(800000, 1)

In [6]:
all_feas = pq.read_table(
    f"preprocessedDataset/{nm}.parquet",
    columns = feas_cols_v2 + [target_label, traceid_col]
).to_pandas()
all_feas.shape

(800000, 51)

In [9]:
if NEED_TO_PREPARE_DATA:
    for i in tqdm.tqdm(range(n_fold)):
        train_lines = all_feas[all_userid[userid_col] % n_fold != i]
        train_lines.to_parquet(f"preprocessedDataset/{nm}-IDResd_Not_{i}.parquet", index=False)
        del train_lines
        gc.collect()

        test_lines = all_feas[all_userid[userid_col] % n_fold == i]
        test_lines.to_parquet(f"preprocessedDataset/{nm}-IDResd_{i}.parquet", index=False)
        del test_lines
        gc.collect()

    del all_feas
    gc.collect()

100%|█████████████████████████████████████████████| 5/5 [00:05<00:00,  1.05s/it]


In [10]:
for i in range(n_fold):
    if not NEED_TO_TRAIN:
        continue
    print("*"*30, f"preparing model for predicting id residual is {i}......")
    all_feas_trn = pq.read_table(
        f"preprocessedDataset/{nm}-IDResd_Not_{i}.parquet",
        columns=feas_cols_v2 + [target_label]
    ).to_pandas()
    print("splitting data...")
    y = all_feas_trn[target_label]
    train, test = train_test_split(all_feas_trn, test_size=0.25, random_state=30, shuffle=True, stratify=y)
    
    print("start training...")
    model_path = train_original_model_1(
        train, test, 
        feas_cols_v2, 
        f"{nm}-IDResd_{i}", ## 模型的名字
        "trained_models",
        target_label,
    )
    
    ## 把特征重要性写到文件里
    importances, sorted_features = get_importance_fixed1(model_path, feas_cols_v2)
    len(importances), len(sorted_features)
    if not os.path.exists("sorted_feas_importance/"):
        os.makedirs("sorted_feas_importance/")
    with open(f"sorted_feas_importance/{nm}-IDResd_{i}--hasImportance_{len(importances)}.txt", "w") as f:
        f.write("\n".join(sorted_features))

****************************** preparing model for predicting id residual is 0......
splitting data...
start training...
[18:43:11] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:576: 
Parameters: { "n_gpus" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.70138	test-auc:0.70088
[9]	train-auc:0.71308	test-auc:0.71168
train time: 3.0662708282470703
****************************** preparing model for predicting id residual is 1......
splitting data...
start training...
[18:43:15] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:576: 
Parameters: { "n_gpus" } might not be used.

  This could be a false alarm, with some parameters getting used by langu

In [8]:
del all_feas_trn, train, test, y
gc.collect()

7